In [16]:
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Check if CUDA is available and print the status
if torch.cuda.is_available():
    print("CUDA is available, using GPU")
    device = torch.device("cuda")
else:
    print("CUDA not available, using CPU")
    device = torch.device("cpu")
# Print the device being used
print(f"Using device: {device}")

CUDA is available, using GPU
Using device: cuda


In [17]:
class CustomDataset(Dataset):
    def __init__(self, path, transform):
        super().__init__()
        self.data = pd.read_csv(path, header='infer').values
        self.length = self.data.shape[0]
        self.transform = transform
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        flatimage = self.data[idx,:-1].astype(np.uint8)
        image = self.transform(np.reshape(flatimage, (128, 128, 3)))
        label = self.data[idx, 49152]
        return image, label

dataset = CustomDataset('dataset.csv', ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [18]:
class CCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 21)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [19]:
model = CCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

In [21]:
n_epochs = 50
for epoch in range(n_epochs):
    loss = train_epoch(train_loader, model, loss_fn, optimizer)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.4f}")

Epoch 1/50, Loss: 2.9553
Epoch 2/50, Loss: 2.3559
Epoch 3/50, Loss: 1.9610
Epoch 4/50, Loss: 1.6503
Epoch 5/50, Loss: 1.4205
Epoch 6/50, Loss: 1.1418
Epoch 7/50, Loss: 0.9121
Epoch 8/50, Loss: 0.6593
Epoch 9/50, Loss: 0.4879
Epoch 10/50, Loss: 0.3258
Epoch 11/50, Loss: 0.1718
Epoch 12/50, Loss: 0.1487
Epoch 13/50, Loss: 0.1766
Epoch 14/50, Loss: 0.1624
Epoch 15/50, Loss: 0.1280
Epoch 16/50, Loss: 0.0823
Epoch 17/50, Loss: 0.0202
Epoch 18/50, Loss: 0.0144
Epoch 19/50, Loss: 0.0189
Epoch 20/50, Loss: 0.0678
Epoch 21/50, Loss: 0.1788
Epoch 22/50, Loss: 0.1019
Epoch 23/50, Loss: 0.0706
Epoch 24/50, Loss: 0.0206
Epoch 25/50, Loss: 0.0671
Epoch 26/50, Loss: 0.0422
Epoch 27/50, Loss: 0.2286
Epoch 28/50, Loss: 0.0857
Epoch 29/50, Loss: 0.0241
Epoch 30/50, Loss: 0.0087
Epoch 31/50, Loss: 0.0024
Epoch 32/50, Loss: 0.0012
Epoch 33/50, Loss: 0.0008
Epoch 34/50, Loss: 0.0006
Epoch 35/50, Loss: 0.0005
Epoch 36/50, Loss: 0.0005
Epoch 37/50, Loss: 0.0004
Epoch 38/50, Loss: 0.0003
Epoch 39/50, Loss: 0.

In [22]:
# Add evaluation function
def evaluate(dataloader, model, loss_fn):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate(test_loader, model, loss_fn)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 3.2940, Test Accuracy: 64.52%
